Text Summarizing

In [4]:
from summarizer import Summarizer
from transformers import logging, AutoConfig, AutoTokenizer, AutoModel
logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore")

# alternate: bert-large-uncased (better summary)
custom_config = AutoConfig.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2')
custom_model = AutoModel.from_pretrained('mrm8488/bert-tiny-finetuned-squadv2', config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [6]:
f = open("sample.txt","r")
full_text = f.read()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)
summarized_text = ''.join(result)
print (summarized_text)

India has a very ancient tradition of art, which has exchanged many influences with the rest of Eurasia, especially in the first millennium, when Buddhist art spread with Indian religions to Central, East and South-East Asia, the last also greatly influenced by Hindu art.[377] Thousands of seals from the Indus Valley Civilization of the third millennium BCE have been found, usually carved with animals, but a few with human figures. The "Pashupati" seal, excavated in Mohenjo-daro, Pakistan, in 1928–29, is the best known.[378][379] After this there is a long period with virtually nothing surviving.[379][380] Almost all surviving ancient Indian art thereafter is in various forms of religious sculpture in durable materials, or coins.


keyword extraction

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chiragmanjeshwar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords
def get_nouns_multipartite(text):
    out=[]
    extractor = pke.unsupervised.MultipartiteRank()
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.load_document(input=text, stoplist=stoplist)
    #    not contain punctuation marks or stopwords as candidates.
    pos = {'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
  
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=100)
    for key in keyphrases:
            out.append(key[0])
    return out
keywords = get_nouns_multipartite(full_text) 
print (keywords)
filtered_keys=[]
for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
        
print (filtered_keys)

['hindu', 'east', 'india', 'south-east asia', 'central', 'ellora', 'karla', 'ajanta', 'jain', 'shiva', 'sarnath', 'mohenjo-daro', 'sanchi', 'bengal school', 'pakistan', 'art', 'pallava', 'chola', 'indus valley civilization', 'bce', 'nataraja', 'ardhanarishvara', 'eurasia', 'pashupati', 'buddhist', 'south', 'calcutta', 'kalighat', 'british', 'western', 'rajputs', 'prana', 'nihâl chand', 'india mauryan', 'gupta', 'ajanta caves', 'eastern india']
['hindu', 'east', 'india', 'south-east asia', 'central', 'mohenjo-daro', 'pakistan', 'art', 'indus valley civilization', 'bce', 'eurasia', 'pashupati', 'buddhist', 'south']


In [17]:
len(filtered_keys)

14